In [ ]:
# Cell 1: Environment Setup & Imports
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Cell 2: Configuration (Optimized)
class Config:
    # Model backbone: RoBERTa-wwm-ext for Chinese
    MODEL_NAME = "hfl/chinese-roberta-wwm-ext"
    
    DATA_PATH = "DATA_PATH" # Modify to the actual data file name and path after parsing. 
    
    # --- Optimization Changes ---
    # 1. Smaller Batch Size for better generalization stability
    BATCH_SIZE = 32 
    
    # 2. Lower Learning Rate: Standard fine-tuning range 
    # The paper's 3e-4 is often too aggressive for full fine-tuning
    LEARNING_RATE = 3e-4 
    
    # 3. More Epochs with Early Stopping logic implies we give it time to converge
    EPOCHS = 10 
    
    MAX_LEN = 256
    GRADIENT_CLIPPING = 1.0
    
    LABEL_MAP = {"Negative": 0, "Neutral": 1, "Positive": 2}
    NUM_CLASSES = 3

config = Config()

# Cell 3: Data Loading & Split
df = pd.read_csv(config.DATA_PATH)
df = df.dropna(subset=['text', 'sentiment'])
df['label'] = df['sentiment'].map(config.LABEL_MAP)
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

# Split by thread_id to prevent context leakage
unique_threads = df['thread_id'].unique()
train_threads, test_threads = train_test_split(unique_threads, test_size=0.2, random_state=42)
train_threads, val_threads = train_test_split(train_threads, test_size=0.1, random_state=42)

train_df = df[df['thread_id'].isin(train_threads)].reset_index(drop=True)
val_df = df[df['thread_id'].isin(val_threads)].reset_index(drop=True)
test_df = df[df['thread_id'].isin(test_threads)].reset_index(drop=True)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Cell 4: Compute Class Weights (Crucial Optimization)
# Calculate weights to penalize the model more for missing minority classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print(f"Class Weights: {class_weights}")

# Cell 5: Dataset Class
class WeiboSentimentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.text = dataframe['text']
        self.targets = dataframe['label']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = str(self.text[index])
        target = self.targets[index]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

tokenizer = BertTokenizer.from_pretrained(config.MODEL_NAME)
train_loader = DataLoader(WeiboSentimentDataset(train_df, tokenizer, config.MAX_LEN), 
                          batch_size=config.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(WeiboSentimentDataset(val_df, tokenizer, config.MAX_LEN), 
                        batch_size=config.BATCH_SIZE, shuffle=False)
test_loader = DataLoader(WeiboSentimentDataset(test_df, tokenizer, config.MAX_LEN), 
                         batch_size=config.BATCH_SIZE, shuffle=False)

# Cell 6: Model Architecture
class FlatRoBERTa(nn.Module):
    def __init__(self, n_classes):
        super(FlatRoBERTa, self).__init__()
        self.bert = BertModel.from_pretrained(config.MODEL_NAME)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        output = self.dropout(pooled_output)
        return self.classifier(output)

model = FlatRoBERTa(config.NUM_CLASSES).to(device)

# Cell 7: Training Setup (Optimized)
# Use the lower learning rate defined in Config
optimizer = AdamW(model.parameters(), lr=config.LEARNING_RATE)

# Use Weighted Loss to handle imbalance
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

total_steps = len(train_loader) * config.EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=int(total_steps * 0.1), num_training_steps=total_steps
)

# Cell 8: Training Loop
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler):
    model.train()
    losses = []
    correct_predictions = 0
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=config.GRADIENT_CLIPPING)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    losses = []
    correct_predictions = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    
    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses), f1_score(all_targets, all_preds, average='macro')

print("Starting Optimized Training...")
best_macro_f1 = 0
for epoch in range(config.EPOCHS):
    train_acc, train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device, scheduler)
    val_acc, val_loss, val_f1 = eval_model(model, val_loader, loss_fn, device)
    print(f"Epoch {epoch+1}/{config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")
    
    if val_f1 > best_macro_f1:
        best_macro_f1 = val_f1
        torch.save(model.state_dict(), 'optimized_flat_roberta.bin')
        print("=> Saved Best Model")

# Cell 9: Final Evaluation
model.load_state_dict(torch.load('optimized_flat_roberta.bin'))
test_acc, test_loss, test_f1 = eval_model(model, test_loader, loss_fn, device)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")
print(f"Final Test Macro F1: {test_f1:.4f}")

# Detailed Report
model.eval()
all_preds = []
all_targets = []
with torch.no_grad():
    for d in test_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

print("\nClassification Report:")
print(classification_report(all_targets, all_preds, target_names=["Negative", "Neutral", "Positive"], zero_division=0))